In [3]:
%%file ./SingleTableSQL/readme.txt
Assignment 1:
1. Create database, calculate the number of email domain name appears in "mbox.txt"
2. Only consider the email at the line which starts with "From:"
3. The schema is |org(TEXT)|count(INTEGER)|

Overwriting ./SingleTableSQL/readme.txt


In [28]:
%%file ./SingleTableSQL/emaildb.py
import sqlite3
connector=sqlite3.connect("emaildb.sqlite");
cur=connector.cursor();
cur.execute("DROP TABLE IF EXISTS Counts");
cur.execute("CREATE TABLE Counts(org TEXT, count INTEGER)");
f=open("./SingleTableSQL/mbox.txt");
for line in f:
    if not line.startswith("From:"): continue;
    words=line.split();
    org=words[1].split("@")[1];
    result=cur.execute("SELECT count FROM Counts WHERE org=?",(org,));
    hit_row=result.fetchone();
    if hit_row!=None:
        cur.execute("UPDATE Counts SET count=count+1 WHERE org=?",(org,));
    else:
        cur.execute("INSERT INTO Counts(org,count) VALUES(?,1)",(org,));
connector.commit();
for line in cur.execute("SELECT * FROM Counts ORDER BY count DESC LIMIT 10"):
    print line[0],line[1];

f.close();
cur.close();

Overwriting ./SingleTableSQL/emaildb.py


In [31]:
# %load ./SingleTableSQL/emaildb.py
import sqlite3
connector=sqlite3.connect("emaildb.sqlite");
cur=connector.cursor();
cur.execute("DROP TABLE IF EXISTS Counts");
cur.execute("CREATE TABLE Counts(org TEXT, count INTEGER)");
f=open("./SingleTableSQL/mbox.txt");
for line in f:
    if not line.startswith("From:"): continue;
    words=line.split();
    org=words[1].split("@")[1];
    result=cur.execute("SELECT count FROM Counts WHERE org=?",(org,));
    hit_row=result.fetchone();
    if hit_row!=None:
        cur.execute("UPDATE Counts SET count=count+1 WHERE org=?",(org,));
    else:
        cur.execute("INSERT INTO Counts(org,count) VALUES(?,1)",(org,));
connector.commit();
for line in cur.execute("SELECT * FROM Counts ORDER BY count DESC LIMIT 10"):
    print line[0],line[1];

f.close();
cur.close();

iupui.edu 536
umich.edu 491
indiana.edu 178
caret.cam.ac.uk 157
vt.edu 110
uct.ac.za 96
media.berkeley.edu 56
ufp.pt 28
gmail.com 25
et.gatech.edu 17


In [8]:
%%file ./MultiTableSQL/readme.txt
Assignment 2:
1. Parse the "Traks" dict in "Library.xml" into database
2. The schema of database is:
    Artist: |id(INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE)|name(TEXT UNIQUE)|
    Genre:  |id(INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE)|name(TEXT UNIQUE)|
    Album:  |id(INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE)|artist_id(INTEGER)|title(TEXT UNIQUE)|
    Track:  |id(INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE)|title(TEXT UNIQUE)|album_id(INTEGER)|
            |genre_id(INTEGER)|len(INTEGER)|rating(INTEGER)|count(INTEGER)|
3. Combine all the table into table:
    |Track|Artist|Album|Genre|

Overwriting ./MultiTableSQL/readme.txt


In [38]:
%%file ./MultiTableSQL/xmldb.py
import xml.etree.ElementTree as ET
import sqlite3

connector=sqlite3.connect('trackdb.sqlite');
cur=connector.cursor();
cur.executescript('''
    DROP TABLE IF EXISTS Artist;
    DROP TABLE IF EXISTS Genre;
    DROP TABLE IF EXISTS Album;
    DROP TABLE IF EXISTS Track;
    CREATE TABLE Artist(
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        name TEXT UNIQUE);
    CREATE TABLE Genre(
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        name TEXT UNIQUE);
    CREATE TABLE Album(
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        artist_id INTEGER,
        title TEXT UNIQUE);
    CREATE TABLE Track(
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        title TEXT UNIQUE,
        album_id INTEGER,
        genre_id INTEGER,
        len INTEGER,
        rating INTEGER,
        count INTEGER);
''');
f=open("./MultiTableSQL/tracks/Library.xml");
XmlContents=ET.parse(f);
def lookup(d,key):
    flag=False;
    for child in d:
        if flag: return child.text;
        if child.tag == "key" and child.text == key:
            flag=True;
    return None;
Tracks=XmlContents.findall("dict/dict/dict");
for Track in Tracks:
    if (lookup(Track,"Track ID") is None): continue;
    
    name = lookup(Track, 'Name');
    artist = lookup(Track, 'Artist');
    album = lookup(Track, 'Album');
    count = lookup(Track, 'Play Count');
    rating = lookup(Track, 'Rating');
    length = lookup(Track, 'Total Time');
    genre = lookup(Track, 'Genre');
    if name is None or artist is None or album is None or genre is None: continue;
    
    cur.execute("INSERT OR IGNORE INTO Artist(name) VALUES(?)",(artist,));
    cur.execute("SELECT id FROM Artist WHERE name = ?",(artist,));
    artist_id=cur.fetchone()[0];
    
    cur.execute("INSERT OR IGNORE INTO Genre(name) VALUES(?)",(genre,));
    cur.execute("SELECT id FROM Genre WHERE name = ?",(genre,));
    genre_id=cur.fetchone()[0];
    
    cur.execute("INSERT OR IGNORE INTO Album(artist_id,title) VALUES(?,?)",(artist_id,album,));
    cur.execute("SELECT id FROM Album WHERE title = ?",(album,));
    album_id=cur.fetchone()[0];
    
    cur.execute("INSERT OR REPLACE INTO Track(title,album_id,genre_id,len,rating,count) VALUES(?,?,?,?,?,?)",(name,album_id,genre_id,length,rating,count));
    
connector.commit();
print "Track|","Artist|","Album|","Genre";
for tuple in cur.execute('''
    SELECT Track.title, Artist.name, Album.title, Genre.name 
    FROM Track JOIN Genre JOIN Album JOIN Artist 
    ON Track.genre_id = Genre.id and Track.album_id = Album.id 
        AND Album.artist_id = Artist.id
    ORDER BY Artist.name,Track.title LIMIT 3
    '''):
    print tuple[0],'|',tuple[1],'|',tuple[2],'|',tuple[3];


Overwriting ./MultiTableSQL/xmldb.py


In [41]:
!python ./MultiTableSQL/xmldb.py

Track| Artist| Album| Genre
Chase the Ace | AC/DC | Who Made Who | Rock
D.T. | AC/DC | Who Made Who | Rock
For Those About To Rock (We Salute You) | AC/DC | Who Made Who | Rock


In [16]:
%%file ./test.py
import re
email="abc@abc.com";
m=re.match("(.*)@(.*)",email);
if m!=None:
    print m.group(1)

Overwriting ./test.py


In [20]:
# %load test.py
import re
email="abc";
m=re.match("(.*)@(.*)",email);
if m!=None:
    print m.group(1)